In [7]:
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torch import nn
from numpy import array
import numpy as np
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

In [8]:
def yield_tokens(data):
    for doc in data:
        yield tokenizer(doc)

In [9]:
# source text
data = """ Jack and Jill went up the hill\n
To fetch a pail of water\n
Jack fell down and broke his crown\n
And Jill came tumbling after\n """

In [11]:
tokenizer = get_tokenizer("basic_english")
encoded = [tokenizer(text) for text in [data]][0]

# MAJOR BREAKTHROUGH HERE TODO LATER, check chapter 20 notebook for reference

vocab_size = len(encoded)
sequences = list()
for i in range(1, len(encoded)):
  sequence = encoded[i-1:i+1]
  sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

vocab = build_vocab_from_iterator(yield_tokens(y))

# encoding
encoded_y = [vocab(tokenizer(doc)) for doc in y]

y_tensor = torch.tensor(encoded_y)
X_tensor = torch.tensor(X)

print(encoded_y,'\n')

y_ohe = nn.functional.one_hot(y_tensor,num_classes=vocab_size)

print(f'y_ohe shape: {y_ohe.shape}')

y_res = torch.reshape(y_ohe,(len(sequences),vocab_size))

print(f'y_res shape: {y_res.shape}')

dataset = TensorDataset(X_tensor, y_res)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

Total Sequences: 24


TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [19]:
class LM(nn.Module):
  def __init__(self,vocab_size):
    super(LM,self).__init__()
    self.embed = nn.Embedding(vocab_size,10)
    self.lstm = nn.LSTM(10,50)
    self.lin1 = nn.Linear(50,vocab_size)
    self.softmax = nn.Softmax(dim=1)
    self.lin2 = nn.Linear(vocab_size,2)

  def forward(self, input):
    embedded = self.embed(input)
    print('\nembedded:\n',embedded.shape,'\n')
    lstm_out, _ = self.lstm(embedded)
    h1 = self.lin1(lstm_out[-1])
    output = self.softmax(h1)
    return output

In [17]:
model = LM(vocab_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 500

# train
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in dataloader:

        # hajtépés és fájdalom
        print(f'X_batch shape: {X_batch.shape}')
        print(f'y_batch shape: {y_batch.shape}')

        outputs = model(X_batch)


        print(f'outputs shape: {outputs.shape}')


        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# eval
with torch.no_grad():
    model.eval()
    X_tensor = dataset.data
    y_tensor = dataset.labels

    outputs = model(X_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_tensor).float().mean()
    print('Accuracy: %f' % (accuracy.item() * 100))

X_batch shape: torch.Size([2, 25])
y_batch shape: torch.Size([2, 25])

embedded:
 torch.Size([2, 25, 10]) 

outputs shape: torch.Size([25, 25])


ValueError: Expected input batch_size (25) to match target batch_size (2).

In [61]:
print(y_ohe.shape,y_ohe)

torch.Size([24, 1, 25]) tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]

In [60]:
y_res = torch.reshape(y_ohe,(24,25))

In [62]:
y_res

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 